### Import data 

In [3]:
import pandas as pd 

# import the journey csv file
journey_may = pd.read_csv(r"./csv/raw/2021-05.csv")
journey_june = pd.read_csv(r"./csv/raw/2021-06.csv")
journey_july = pd.read_csv(r"./csv/raw/2021-07.csv")

# add month column to the dataframe
journey_may['month'] = 'May'
journey_june['month'] = 'June'
journey_july['month'] = 'July'

# merge all journey into one df 
journey_all = pd.concat([journey_may, journey_june, journey_july])

# import the stations csv file 
stations = pd.read_csv(r".\csv\raw\Helsingin_ja_Espoon_kaupunkipyasemat_avoin.csv")

### Visualize data

In [4]:
shape = journey_all.shape
total_journeys = shape[0]-1 # -1 because of the header
print("Total journeys:", total_journeys)
stations.head()

Total journeys: 3247001


,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551


In [5]:
shape = stations.shape
total_stations = shape[0]-1 # -1 because of the header
print("Total stations:", total_stations)
journey_all.head()


Total stations: 456


,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),month
0,2021-05-31T23:57:25,2021-06-01T00:05:46,94,Laajalahden aukio,100,Teljäntie,2043.0,500,May
1,2021-05-31T23:56:59,2021-06-01T00:07:14,82,Töölöntulli,113,Pasilan asema,1870.0,611,May
2,2021-05-31T23:56:44,2021-06-01T00:03:26,123,Näkinsilta,121,Vilhonvuorenkatu,1025.0,399,May
3,2021-05-31T23:56:23,2021-06-01T00:29:58,4,Viiskulma,65,Hernesaarenranta,4318.0,2009,May
4,2021-05-31T23:56:11,2021-06-01T00:02:02,4,Viiskulma,65,Hernesaarenranta,1400.0,350,May


### Check if all the stations are in the journey

In [6]:
# get the ID colum from all stations 
all_stations_id = stations.iloc[:,1]
all_departure_stations_id = journey_all.iloc[:,2]
all_return_stations_id = journey_all.iloc[:,4]

# make a list of all 
all_stations_id_list = all_stations_id.tolist()
all_departure_stations_id_list = all_departure_stations_id.tolist()
all_return_stations_id_list = all_return_stations_id.tolist()

deparature_station_not_found = [i for i in all_departure_stations_id_list if i not in all_stations_id_list]
return_station_not_found = [i for i in all_return_stations_id_list if i not in all_stations_id_list]

merged_not_found = list(set(deparature_station_not_found + return_station_not_found))

merged_not_found

[754, 997, 999]

Looks like there are some stations that are in the journey, but not in the stations list. 

Namely, the following stations: `754`, `997` and `999`.

It doesn't really matter, since we are not going to query the data for stations in terms of the journey, but it is worth noting.

### Validation: Journey that lasted less than 10 seconds and covered distance less than 10m 

In [7]:
# remove rows from journey_all where the duration is less than 10 and covered_distance is less than 10
journey_all = journey_all[journey_all.iloc[:, 6] > 10]
journey_all = journey_all[journey_all.iloc[:, 7] > 10]

# visualize the journey_all df
shape = journey_all.shape
new_total_journeys = shape[0]-1
print("Total journeys:", new_total_journeys)
print("Journeys invalidated:", total_journeys - new_total_journeys)
journey_all.head()

Total journeys: 3126465
Journeys invalidated: 120536


,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),month
0,2021-05-31T23:57:25,2021-06-01T00:05:46,94,Laajalahden aukio,100,Teljäntie,2043.0,500,May
1,2021-05-31T23:56:59,2021-06-01T00:07:14,82,Töölöntulli,113,Pasilan asema,1870.0,611,May
2,2021-05-31T23:56:44,2021-06-01T00:03:26,123,Näkinsilta,121,Vilhonvuorenkatu,1025.0,399,May
3,2021-05-31T23:56:23,2021-06-01T00:29:58,4,Viiskulma,65,Hernesaarenranta,4318.0,2009,May
4,2021-05-31T23:56:11,2021-06-01T00:02:02,4,Viiskulma,65,Hernesaarenranta,1400.0,350,May


Looks like like we got rid of good number of invalidate journeys. I have manually checked the data in MS Excel that the journeys are valid.


### Get rid of rows that are not necessary for the assingment and name everything in good old English

In [8]:
# stations df 
# remove unnecessary columns
stations = stations.drop(columns=["FID", "Nimi", "Namn", "Adress", "Stad"])
# rename the columns
stations.columns = ["id", "name", "address", "city", "operator", "capacity", "lon", "lat"]
stations.head()

,id,name,address,city,operator,capacity,lon,lat
0,501,Hanasaari,Hanasaarenranta 1,Espoo,CityBike Finland,10,24.840319,60.165820
1,503,Keilalahti,Keilalahdentie 2,Espoo,CityBike Finland,28,24.827467,60.171524
2,505,Westendinasema,Westendintie 1,Espoo,CityBike Finland,16,24.805758,60.168266
3,507,Golfpolku,Golfpolku 3,Espoo,CityBike Finland,16,24.796136,60.168143
4,509,Revontulentie,Revontulentie 10,Espoo,CityBike Finland,30,24.802938,60.171551


In [9]:
# journey_all df
# rename the columns
journey_all.columns = ["departure_time", "return_time", "departure_station_id", "departure_station", "return_station_id", "return_station", "duration", "covered_distance", "month"]
# add a column to journey_all df called id that increases by 1 each time
journey_all["id"] = journey_all.index

journey_all.head()

,departure_time,return_time,departure_station_id,departure_station,return_station_id,return_station,duration,covered_distance,month,id
0,2021-05-31T23:57:25,2021-06-01T00:05:46,94,Laajalahden aukio,100,Teljäntie,2043.0,500,May,0
1,2021-05-31T23:56:59,2021-06-01T00:07:14,82,Töölöntulli,113,Pasilan asema,1870.0,611,May,1
2,2021-05-31T23:56:44,2021-06-01T00:03:26,123,Näkinsilta,121,Vilhonvuorenkatu,1025.0,399,May,2
3,2021-05-31T23:56:23,2021-06-01T00:29:58,4,Viiskulma,65,Hernesaarenranta,4318.0,2009,May,3
4,2021-05-31T23:56:11,2021-06-01T00:02:02,4,Viiskulma,65,Hernesaarenranta,1400.0,350,May,4


### Save the clean data as two csv files 

In [10]:
# stations df
# save as csv file
stations.to_csv(r"./csv/validated/stations.csv", index=False)
# save as json file
stations.to_json(r"./csv/validated/stations.json", orient="records")
# journey_all df
# save as csv file
journey_all.to_csv(r"./csv/validated/journey_all.csv", index=False)
# save as json file
journey_all.to_json(r"./csv/validated/journey_all.json", orient="records")
